# WeatherPy - Liberty Heise

Analysis:

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 640


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [3]:
# Set the API base URL
url = "https://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []
units = "metric"
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities[:50]):    #DEBUG
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0
    city_url = f"{url}appid={weather_api_key}&units={units}&q={city.replace(' ','+')}"
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    record_count += 1
    try:
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

    #Append the City information into city_data list
        city_data.append({"City": city, 
                            "Lat": city_lat, 
                            "Lng": city_lng, 
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Country": city_country,
                            "Date": city_date})
        #print(f"{city}{len(city_data)}")

    except:
        print(f"City {city} not found. Skipping...")
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | orahovac
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | margate
Processing Record 4 of Set 1 | mataura
Processing Record 5 of Set 1 | hermanus
Processing Record 6 of Set 1 | el porvenir
Processing Record 7 of Set 1 | carnarvon
Processing Record 8 of Set 1 | taolanaro
City taolanaro not found. Skipping...
Processing Record 9 of Set 1 | vila velha
Processing Record 10 of Set 1 | palabuhanratu
City palabuhanratu not found. Skipping...
Processing Record 11 of Set 1 | bengkulu
Processing Record 12 of Set 1 | ushuaia
Processing Record 13 of Set 1 | butaritari
Processing Record 14 of Set 1 | jamestown
Processing Record 15 of Set 1 | berbera
Processing Record 16 of Set 1 | auki
Processing Record 17 of Set 1 | ostrovnoy
Processing Record 18 of Set 1 | kenora
Processing Record 19 of Set 1 | albany
Processing Record 20 of Set 1 | hasaki
Processing Record 21 of Set 1 | menongue
Proce

In [4]:
city_data

[{'City': 'orahovac',
  'Lat': 42.3994,
  'Lng': 20.6547,
  'Max Temp': 13.92,
  'Humidity': 69,
  'Cloudiness': 95,
  'Wind Speed': 3.96,
  'Country': 'XK',
  'Date': 1667584732},
 {'City': 'busselton',
  'Lat': -33.65,
  'Lng': 115.3333,
  'Max Temp': 10.72,
  'Humidity': 90,
  'Cloudiness': 2,
  'Wind Speed': 4.21,
  'Country': 'AU',
  'Date': 1667584733},
 {'City': 'margate',
  'Lat': 51.3813,
  'Lng': 1.3862,
  'Max Temp': 10.39,
  'Humidity': 73,
  'Cloudiness': 13,
  'Wind Speed': 2.24,
  'Country': 'GB',
  'Date': 1667584734},
 {'City': 'mataura',
  'Lat': -46.1927,
  'Lng': 168.8643,
  'Max Temp': 7.84,
  'Humidity': 68,
  'Cloudiness': 96,
  'Wind Speed': 4.47,
  'Country': 'NZ',
  'Date': 1667584735},
 {'City': 'hermanus',
  'Lat': -34.4187,
  'Lng': 19.2345,
  'Max Temp': 19.33,
  'Humidity': 86,
  'Cloudiness': 17,
  'Wind Speed': 5.82,
  'Country': 'ZA',
  'Date': 1667584737},
 {'City': 'el porvenir',
  'Lat': 13.7617,
  'Lng': -87.3458,
  'Max Temp': 27.63,
  'Humidity':

In [5]:
# Convert the cities weather data into a Pandas DataFrame

weather_data_df = pd.DataFrame(city_data)
# Show Record Count


In [6]:
# Display sample data
weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,orahovac,42.3994,20.6547,13.92,69,95,3.96,XK,1667584732
1,busselton,-33.6500,115.3333,10.72,90,2,4.21,AU,1667584733
2,margate,51.3813,1.3862,10.39,73,13,2.24,GB,1667584734
3,mataura,-46.1927,168.8643,7.84,68,96,4.47,NZ,1667584735
4,hermanus,-34.4187,19.2345,19.33,86,17,5.82,ZA,1667584737
5,el porvenir,13.7617,-87.3458,27.63,76,100,2.08,HN,1667584737
6,carnarvon,-24.8667,113.6333,19.33,74,0,10.34,AU,1667584738
7,vila velha,-20.3297,-40.2925,23.97,67,75,9.26,BR,1667584739
8,bengkulu,-3.8004,102.2655,23.94,90,100,1.82,ID,1667584690
9,ushuaia,-54.8000,-68.3000,9.79,76,100,4.12,AR,1667584741


In [7]:
# Export the City_Data into a csv
weather_data_df.to_csv("output_data/cities.csv", index_label="City_ID")


In [8]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,orahovac,42.3994,20.6547,13.92,69,95,3.96,XK,1667584732
1,busselton,-33.6500,115.3333,10.72,90,2,4.21,AU,1667584733
2,margate,51.3813,1.3862,10.39,73,13,2.24,GB,1667584734
3,mataura,-46.1927,168.8643,7.84,68,96,4.47,NZ,1667584735
4,hermanus,-34.4187,19.2345,19.33,86,17,5.82,ZA,1667584737


### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
lat_temp = city_data.loc[city_data["lat"]]
x_values = city_data["Lat"]
y_values = city_data["Max Temp"]

plt.xlabel ("Lat")
plt.ylabel ("Max Temp")
plt.title ("City Max Latitude vs. Temperature")
plt.show
# Save the figure
plt.savefig("output_data/Fig1.png")



#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
x_values = city_data["Lat"]
y_values = city_data["Humidity"]

plt.xlabel("Lat")
plt.ylabel("Humidity")
plt.title("City Latitude vs. Humidity")
plt.show()

# Save the figure
plt.savefig("output_data/Fig2.png")



#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
x_values = city_data["Lat"]
y_values = city_data["Cloudiness"]

plt.xlabel("Lat")
plt.ylabel("Cloudiness")
plt.title("City Latitude vs. Cloudiness")
plt.show()

# Save the figure
plt.savefig("output_data/Fig3.png")


#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
x_values = city_data["Lat"]
y_values = city_data["Wind Speed"]

plt.scatter(x_values, y_values)

plt.xlabel("Lat")
plt.ylabel("Wind Speed")
plt.title("City Latitude vs. Wind Speed")
plt.show()

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
# YOUR CODE HERE

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
# YOUR CODE HERE

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
# YOUR CODE HERE

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Max Temp"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Max Temp")
plt.show()

In [ ]:
# Linear regression on Southern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Max Temp"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Max Temp")
plt.show()

**Discussion about the linear relationship:** 
As one moves closer to the equator (lat is closer to zero)the temperature increases.

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Humidity"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Humidity")
plt.show()

In [ ]:
# Southern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Humidity"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Humidity")
plt.show()

**Discussion about the linear relationship:** 
There is no direct correlation between latitidue and humidity.

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Max Temp"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("")
plt.ylabel("")
plt.show()

In [ ]:
# Southern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Cloudiness"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Cloudiness")
plt.show()

**Discussion about the linear relationship:** 
Cloudiness increases as one moves closer to the equator.

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Wind Speed"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Wind Speed")
plt.show()

In [ ]:
# Southern Hemisphere
x_values = city_data["Lat"]
y_values = city_data["Wind Speed"]
(slope, intercept, rvalue, pvalue, stdevr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eg = "y = " + str(round(slope,2)) + ("x = "+ str(round(intercept,2)))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eg, (5.8,0.8),fontsize = 15, color ="red")
plt.xlabel("Lat")
plt.ylabel("Wind Speed")
plt.show()

**Discussion about the linear relationship:** 
Wind Speed slightly decreases as one moves closer to the equator.